### C. Hydrogen solubility and diffusivity in bulk α-iron

In [65]:
import pyiron
import ase.calculators
import ase.optimize
import ase.neb
import ase.visualize
import matplotlib.pyplot as plt
import numpy as np
import scipy.optimize

In [2]:
potential =  'FeH-NNIP'

1. solution energy

$$E_s = E_{Fe+H} - E_{Fe}- E_{H_2}/2$$

$E_{H_2}/2$: the energy of a gaseous $H_2$ molecule

In [22]:
# calculation of E_Fe
pr = pyiron.Project('solution_E')
job = pr.create.job.Lammps('bulk', delete_existing_job=True)
bulk = pr.create.structure.bulk('Fe', cubic=True).repeat([3,3,3])
job.structure = bulk
job.potential = potential
job.calc_minimize(pressure=0.0)
job.run(delete_existing_job=True)

The job bulk was saved and received the ID: 10592


In [23]:
E_Fe = job.output.energy_tot[-1]
bulk = job.get_structure(-1)
cell = bulk.get_cell()

In [37]:
# H at O-site
bulk_H_O = bulk.copy()
H = pr.create.structure.atoms('H', positions=[(2.83/2, 2.83, 2.83/2)], cell=cell)
bulk_H_O += H
bulk_H_O.plot3d()

NGLWidget()

In [38]:
job = pr.create.job.Lammps('bulk_H_O', delete_existing_job=True)
job.structure = bulk_H_O
job.potential = potential
job.calc_minimize()
job.run(delete_existing_job=True)

The job bulk_H_O was saved and received the ID: 10595


In [39]:
E_Fe_H_O = job.output.energy_tot[-1]
E_Fe_H_O

-448.064760912784

In [40]:
E_Fe_H_O - E_Fe

-3.021180453103966

In [41]:
# H at T-site

bulk_H_T = bulk.copy()
H = pr.create.structure.atoms('H', positions=[(2.83/4*6, 2.83/4*8, 2.83/4*6-2.83/4)], cell=cell)
bulk_H_T += H
bulk_H_T.plot3d()

NGLWidget()

In [42]:
job = pr.create.job.Lammps('bulk_H_T', delete_existing_job=True)
job.structure = bulk_H_T
job.potential = potential
job.calc_minimize()
job.run(delete_existing_job=True)

The job bulk_H_T was saved and received the ID: 10596


In [43]:
E_Fe_H_T = job.output.energy_tot[-1]
E_Fe_H_T

-448.175938372699

In [51]:
E_Fe_H_T -E_Fe_H_O

-0.1111774599150408

In [46]:
0.236 - 0.389
# E difference on paper

-0.15300000000000002

In [105]:
job = pr.load('bulk_H_T')
job.animate_structures()

NGLWidget(max_frame=1)

In [106]:
job.output.energy_tot

array([-447.9359933 , -448.17593837])

? \
$E_{Fe+H}$ = ...\
after inserting H, $E_{Fe+H}$ should be the energy before or after relaxation\
$E_{H_2}$ ~ 4.5 eV?

In [107]:
job = pr.load('bulk_H_O')
E_H_O_ms = job.output.energy_tot[0]
job = pr.load('bulk_H_T')
E_H_T_ms = job.output.energy_tot[-1]

E_H_T_ms - E_H_O_ms

-0.75238956341002

In [101]:
# print(E_Fe_H_T -  E_Fe)
# print(E_Fe_H_O -  E_Fe)
# E_Fe_H_T - E_Fe_H_O

2. Diffusion energy barrier (T to T)

In [60]:
bulk_H_T_imageI = bulk.copy()
H = pr.create.structure.atoms('H', positions=[(2.83/4*6, 2.83/4*8, 2.83/4*6-2.83/4)], cell=cell)
bulk_H_T_imageI += H
bulk_H_T_imageI.plot3d()

NGLWidget()

In [61]:
bulk_H_T_imageF = bulk.copy()
H = pr.create.structure.atoms('H', positions=[(2.83/4*6, 2.83/4*8, 2.83/4)], cell=cell)
bulk_H_T_imageF += H
bulk_H_T_imageF.plot3d()

NGLWidget()

In [63]:
IS = bulk_H_T_imageI.to_ase()
FS = bulk_H_T_imageF.to_ase()

LAMMPS calculator:

In [102]:
from ase import Atom, Atoms
from ase.build import bulk
from ase.calculators.lammpsrun import LAMMPS

parameters = {'pair_style': 'eam/alloy',
            'pair_coeff': ['* * NiAlH_jea.eam.alloy H Al']}

files = ['NiAlH_jea.eam.alloy']

Ni = bulk('Ni', cubic=True)
H = Atom('H', position=Ni.cell.diagonal()/2)
NiH = Ni + H

lammps = LAMMPS(parameters=parameters, files=files)

NiH.calc = lammps
NiH.get_potential_energy()

/home/gu.huang/miniconda3/envs/pyiron/lib/python3.12/site-packages/ase/calculators/lammpsrun.py:191: UserWarning: You are using an old syntax to set 'parameters'.
Please use LAMMPSRUN.set().
  warnings.warn(self.legacy_warn_string.format("parameters"))


RuntimeError: Failed to retrieve any thermo_style-output

https://wiki.fysik.dtu.dk/ase/ase/calculators/lammpsrun.html#lammps

?

3. Diffusivity of H atoms (see diffusivity and C)